## Installations

PS: This recipe is not working. More info here:
https://github.com/langchain-ai/langchain-weaviate/issues/207

In [1]:
%pip install -U weaviate-client llama-index llama-index-vector-stores-weaviate llama-index-embeddings-openai llama-index-readers-web


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Connect to Weaviate

In [2]:
import os
import weaviate
from weaviate import classes as wvc
client = weaviate.connect_to_embedded(
    headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY"]
    }
)

{"action":"startup","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-12-26T20:43:46-03:00"}
{"action":"startup","auto_schema_enabled":true,"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-12-26T20:43:46-03:00"}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-12-26T20:43:46-03:00"}
{"build_git_commit":"ab0312d5d","build_go_versio

In [3]:
print("Client Version:", weaviate.__version__, "Server Version:", client.get_meta().get("version"))

Client Version: 4.10.2 Server Version: 1.26.6


{"action":"hnsw_prefill_cache_async","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-12-26T20:43:49-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"Completed loading shard blogpost_n16RYKohRyTa in 26.0895ms","time":"2024-12-26T20:43:49-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","count":3000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-12-26T20:43:49-03:00","took":70791}


## Create Schema

In [4]:
client.collections.delete("BlogPost")
collection = client.collections.create(
    name="BlogPost",
    description="Blog Post from Weaviate website",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
            model="text-embedding-3-small"
        )
)

## Load in Data

In [5]:
from llama_index.readers.web import SimpleWebPageReader

loader = SimpleWebPageReader(html_to_text=True)
blog = loader.load_data(urls=['https://weaviate.io/blog/llamaindex-and-weaviate'])

{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/blogpost/pCCvvGK0UfZZ/proplengths does not exist, creating new tracker","time":"2024-12-26T20:43:51-03:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-12-26T20:43:51-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"Created shard blogpost_pCCvvGK0UfZZ in 1.077625ms","time":"2024-12-26T20:43:51-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":

In [6]:
from weaviate.util import generate_uuid5
# we need to have uuid in places
# lets generate
for d in blog:
    d.id_ = generate_uuid5(d.id_)

In [7]:
# an example of the data
blog[0]

Document(id_='30140a72-a9c9-50df-9201-a40ee5ce0063', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='[← Back to Blogs](/blog)\n\nSkip to main content\n\n[Ready to level up your AI tech stack? Read GigaOm\'s Sonar Report for Vector\nDatabases](/reports/gigaom-sonar-report-for-vector-databases)\n\n[![Weaviate](/img/site/weaviate-logo-horizontal-\nlight-1.svg)![Weaviate](/img/site/weaviate-logo-horizontal-\ndark-1.svg)****](/)[.](/developers/weaviate/api/rest)\n\nProduct\n\n  *     * Overview\n    * [Vector Database](/platform)\n    * [Workbench](/workbench)\n    * [Integrations](/product/integrations)\n\n    * Deployment\n    * [Serverless Cloud](/deployment/serverless)\n    * [Enterprise Cloud](/deployment/enterprise-cloud)\n    * [Bring Your Own Cloud](/deployment/byoc)\n    * [Enablement](/deployment/e

## Construct Vector Store

In [8]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.storage.storage_context import StorageContext
import os
import openai

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

embed_model = OpenAIEmbedding(
    embed_batch_size=10,
    model="text-embedding-3-small"
)
openai.api_key = os.environ["OPENAI_APIKEY"]
Settings.embed_model = embed_model

# construct vector store
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="BlogPost"
)

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# set up the index
index = VectorStoreIndex(blog, storage_context=storage_context)

query_engine = index.as_query_engine()

In [9]:
response = query_engine.query("What is Llamaindex?")
print(response)

LlamaIndex is a data framework for building large language model (LLM) applications. It provides tools for data ingestion, management, and querying of external data to be used with LLM applications.


## Close the sync client and get an async client

because we are using the embedded weaviate, we need to close the client. If using a remote weaviate, this is not needed.


In [10]:

client.close()

{"action":"restapi_management","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","docker_image_tag":"localhost","level":"info","msg":"Shutting down... ","time":"2024-12-26T20:44:12-03:00"}
{"action":"restapi_management","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","docker_image_tag":"localhost","level":"info","msg":"Stopped serving weaviate at http://127.0.0.1:8079","time":"2024-12-26T20:44:12-03:00"}
{"action":"telemetry_push","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"telemetry terminated","payload":"\u0026{MachineID:4ef17d4a-66d1-4cb7-b0ce-063941a4c062 Type:TERMINATE Version:1.26.6 NumObjects:0 OS:darwin Arch:arm64 UsedModules:[text2vec-openai]}","time":"2024-12-26T20:44:13-03:00"}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","bu

## Query Time

In [11]:

import asyncio
import weaviate
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

import nest_asyncio
nest_asyncio.apply()

async def subquestion():
    import weaviate

    async with weaviate.use_async_with_embedded(version="1.28.2") as async_client:
        # The async context manager automatically connects and disconnects
        # Use the async client - for example, check if it's ready
        

        vector_store = WeaviateVectorStore(
            weaviate_client=async_client, index_name="BlogPost"
        )

        loaded_index = VectorStoreIndex.from_vector_store(vector_store)
        query_engine = loaded_index.as_query_engine()
        #response = await query_engine.aquery('How does LlamaIndex help data indexing in Weaviate?')
        query_engine_tools = [
            QueryEngineTool(
                query_engine = query_engine,
                metadata = ToolMetadata(name='BlogPost', description='Blog post about the integration of LlamaIndex and Weaviate')
            )
        ]
        query_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)
        response = query_engine.query('How does LlamaIndex help data indexing in Weaviate?')
    return response

response = await subquestion()

{"build_git_commit":"5a3991d2d4","build_go_version":"go1.23.4","build_image_tag":"HEAD","build_wv_version":"1.28.2","level":"warning","log_level_env":"","msg":"log level not recognized, defaulting to info","time":"2024-12-26T20:44:21-03:00"}
{"action":"startup","build_git_commit":"5a3991d2d4","build_go_version":"go1.23.4","build_image_tag":"HEAD","build_wv_version":"1.28.2","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-12-26T20:44:21-03:00"}
{"action":"startup","auto_schema_enabled":true,"build_git_commit":"5a3991d2d4","build_go_version":"go1.23.4","build_image_tag":"HEAD","build_wv_version":"1.28.2","level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-12-26T20:44:21-03:00"}
{"build_git_commit":"5a3991d2d4","build_go_version":"go1.23.4","build_image_tag":"HEAD","build_wv_version":"1.

**********
Trace: index_construction
**********


{"action":"hnsw_prefill_cache_async","build_git_commit":"5a3991d2d4","build_go_version":"go1.23.4","build_image_tag":"HEAD","build_wv_version":"1.28.2","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-12-26T20:44:25-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"5a3991d2d4","build_go_version":"go1.23.4","build_image_tag":"HEAD","build_wv_version":"1.28.2","level":"info","msg":"Completed loading shard blogpost_pCCvvGK0UfZZ in 9.021458ms","time":"2024-12-26T20:44:25-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"5a3991d2d4","build_go_version":"go1.23.4","build_image_tag":"HEAD","build_wv_version":"1.28.2","count":3000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-12-26T20:44:25-03:00","took":32375}
/Users/dudanogueira/dev/weaviate/lab/.venv/lib/python3.12/site-packages/pydantic/main.py:1355: PydanticDeprecatedSince20: The `schema` method is deprecated; use `

Generated 3 sub questions.
[BlogPost] Q: What is LlamaIndex?
[BlogPost] Q: What is Weaviate?
[BlogPost] Q: How does LlamaIndex integrate with Weaviate?
[BlogPost] A: Weaviate is a vector database that acts as an external storage provider. It is a critical component that, when combined with LlamaIndex, allows for the easy setup of a powerful and reliable retrieval-augmented generation (RAG) stack. Weaviate enables the orchestration of interactions with language models and external storage providers to create a seamless "chat with your data" experience.
[BlogPost] A: LlamaIndex is a data framework for building large language model (LLM) applications. It provides tools for data ingestion, management, and querying of external data to be used with LLM applications. The framework offers connectors to various data sources, supports indexing of unstructured, semi-structured, and structured data, and provides a query engine for advanced retrieval and query tasks over the data.
[BlogPost] A: Lla

{"action":"restapi_management","build_git_commit":"5a3991d2d4","build_go_version":"go1.23.4","build_image_tag":"HEAD","build_wv_version":"1.28.2","level":"info","msg":"Shutting down... ","time":"2024-12-26T20:44:31-03:00","version":"1.28.2"}
{"action":"restapi_management","build_git_commit":"5a3991d2d4","build_go_version":"go1.23.4","build_image_tag":"HEAD","build_wv_version":"1.28.2","level":"info","msg":"Stopped serving weaviate at http://127.0.0.1:8079","time":"2024-12-26T20:44:31-03:00","version":"1.28.2"}
{"action":"telemetry_push","build_git_commit":"5a3991d2d4","build_go_version":"go1.23.4","build_image_tag":"HEAD","build_wv_version":"1.28.2","level":"info","msg":"telemetry terminated","payload":"\u0026{MachineID:70b07106-62fd-4b46-a21e-9e8eb03ef05d Type:TERMINATE Version:1.28.2 NumObjects:1 OS:darwin Arch:arm64 UsedModules:[text2vec-openai]}","time":"2024-12-26T20:44:32-03:00"}
{"build_git_commit":"5a3991d2d4","build_go_version":"go1.23.4","build_image_tag":"HEAD","build_wv_ver

In [12]:
response.response

'LlamaIndex assists in data indexing in Weaviate by providing tools for data ingestion, management, and querying of external data. This integration allows for the setup of a powerful and reliable retrieval-augmented generation (RAG) stack, enabling interactions with language models and external storage providers to create seamless experiences like search engines and chatbots over data.'

In [13]:
# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.core.callbacks import CBEventType, EventPayload

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

Sub Question 0: What is LlamaIndex?
Answer: LlamaIndex is a data framework for building large language model (LLM) applications. It provides tools for data ingestion, management, and querying of external data to be used with LLM applications. The framework offers connectors to various data sources, supports indexing of unstructured, semi-structured, and structured data, and provides a query engine for advanced retrieval and query tasks over the data.
Sub Question 1: What is Weaviate?
Answer: Weaviate is a vector database that acts as an external storage provider. It is a critical component that, when combined with LlamaIndex, allows for the easy setup of a powerful and reliable retrieval-augmented generation (RAG) stack. Weaviate enables the orchestration of interactions with language models and external storage providers to create a seamless "chat with your data" experience.
Sub Question 2: How does LlamaIndex integrate with Weaviate?
Answer: LlamaIndex integrates with Weaviate by usi